In [0]:
# 1) (Opcional) Verificar se o cluster está ativo: apenas rode uma célula simples
#spark.range(1).count()

# 2) Criar (ou garantir) uma pasta de trabalho em DBFS (se quiser listar algo local)
dbutils.fs.mkdirs("dbfs:/tmp/dados")  # não é um mount, só diretório no DBFS do workspace

tenant_id     = "dc61d494-96b7-497b-938a-24df549598e0"   # Directory (tenant) ID
client_id     = "90966096-825b-453e-b8ef-4649b55118e5"   # Application (client) ID
client_secret = dbutils.secrets.get(scope="pri-scope", key="service-credential-key-name")

account  = "datalakepriscylla"   # sua storage account
container = "imoveis"            # CONFIRA se esse container existe no seu account!
root = f"abfss://{container}@{account}.dfs.core.windows.net"

endpoint = f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"

spark.conf.set(f"fs.azure.account.auth.type.{account}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{account}.dfs.core.windows.net",
               "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{account}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{account}.dfs.core.windows.net", client_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{account}.dfs.core.windows.net", endpoint)


In [0]:
display(dbutils.fs.ls(f"{root}/"))   # lista o topo do container


In [0]:
# escrever um dataset de teste
test_write = f"{root}/bronze/_smoke_test/"
spark.range(1, 6).write.mode("overwrite").parquet(test_write)

In [0]:
# conferir que escreveu
display(dbutils.fs.ls(test_write))

# ler de volta
df = spark.read.parquet(test_write)
df.show()